#### Inputs and Outputs

This might seem odd that we include the inputs as a description of machine learning algorthm but this is very important. The first point to mention is that each machine learning algorithm is for a specific type of output: quantitative or qualitative. Certain machine learning algorithms are specifically for qualitative outputs and some for quantitative (and some for both). Linear regression (and regression tasks in general) are for quantitative outcomes. In future lectures we will talk about ML for qualitative outcomes.

The second concern is about the inputs. Some models need very specific inputs. So for example, or linear regression can only take quantitative features, so we need to make sure to convert all the values to quantitative ones before the regression. The second point that needs to be made is whether the data needs to be standardized. For linear regression the answer is two parts: no, unless there are interaction or quadratic terms (which you can read about [here](http://www.stat.columbia.edu/~gelman/research/published/ape17.pdf).

In summary two points:

1. Linear Regression is a regression model